In [1]:
import logging
from importlib import reload
reload(logging)
logger = logging.getLogger(__name__)

In [2]:
import pandas as pd
import time
import random
import os
from pathlib import Path
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import load_prompt
from pprint import pprint
from pathlib import Path
import numpy as np
from llm_assessor import (
    run_prompt_chain,
    extract_marks_from_graded_answer,
)

load_dotenv(override=True)

ERROR:root:Failed to update token costs. Using static costs.
/Users/junaid/.virtualenvs/audenshaw_exam_validation/lib/python3.11/site-packages/tokencost/constants.py:61: RuntimeWarning: coroutine 'update_token_costs' was never awaited
  TOKEN_COSTS = TOKEN_COSTS_STATIC


True

In [3]:
# Set working directory - Not required if using Jupyter outside of VScode
workdir = os.environ["workdir"]
os.chdir(workdir)
os.getcwd()

'/Users/junaid/Developer/audenshaw_exam_validation'

In [4]:
# Load Directories containing Prompt Templates and Student Answers
prompt_dir = Path("prompt_templates")

data_dir = Path(f"{workdir}/validation_results/processed_data")

In [5]:
# Load LLM
llm = ChatOpenAI(
    model_name="gpt-4o", 
    temperature=0.00, 
    max_tokens=1000,
    )

In [18]:
# logger = logging.getLogger()
# logger.setLevel(level=logging.INFO)
# formatter = logging.Formatter('%(asctime)s:%(levelname)s:%(message)s')

# # create console handler with a higher log level
# ch = logging.StreamHandler()
# ch.setLevel(logging.INFO)
# ch.setFormatter(formatter)

# logger.addHandler(ch)
root = logging.getLogger()
for handler in root.handlers[:]:
    root.removeHandler(handler)

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
    )


# Load in Student Answers

In [21]:
# Load in Data
completed_paper_df = pd.read_csv(data_dir / "student_answers_augmented.csv")
student_answers_records = completed_paper_df.to_dict(orient="records")

# Read in Prompt Templates

In [22]:
# Load all prompt templates
prompts = {
    "grade_answer": 
    {
        "aqa_history": {
            "hs_analyse": load_prompt(prompt_dir/ "aqa_history" / "analyse_prompt" / "grade_answer_aqa_history_analyse_prompt.json"),
            "hs_explain": load_prompt(prompt_dir/ "aqa_history" / "explain_prompt" / "grade_answer_aqa_history_explain_prompt.json"),
            "hs_judgement": load_prompt(prompt_dir/ "aqa_history" / "judgement_prompt" / "grade_answer_aqa_history_judgement_prompt.json"),
            "hs_spag": load_prompt(prompt_dir/ "aqa_history" / "spag_prompt" / "grade_answer_aqa_history_spag_prompt.json")
        },
        "edexcel_business_studies": {
            "bs_analyse": load_prompt(prompt_dir/ "edexcel_business_studies" / "analyse_prompt" / "grade_answer_edexcel_business_studies_analyse_prompt.json"),
            "bs_discuss": load_prompt(prompt_dir/ "edexcel_business_studies" / "discuss_prompt" / "grade_answer_edexcel_business_studies_discuss_prompt.json"),
            "bs_evaluate": load_prompt(prompt_dir/ "edexcel_business_studies" / "evaluate_prompt" / "grade_answer_edexcel_business_studies_evaluate_prompt.json"),
            "bs_explain": load_prompt(prompt_dir/ "edexcel_business_studies" / "explain_prompt" / "grade_answer_edexcel_business_studies_explain_prompt.json"),
            "bs_identify": load_prompt(prompt_dir/ "edexcel_business_studies" / "identify_prompt" / "grade_answer_edexcel_business_studies_identify_prompt.json"),
            "bs_justify": load_prompt(prompt_dir/ "edexcel_business_studies" / "justify_prompt" / "grade_answer_edexcel_business_studies_justify_prompt.json"),
            "bs_outline": load_prompt(prompt_dir/ "edexcel_business_studies" / "outline_prompt" / "grade_answer_edexcel_business_studies_outline_prompt.json"),
            "bs_state": load_prompt(prompt_dir/ "edexcel_business_studies" / "state_prompt" / "grade_answer_edexcel_business_studies_state_prompt.json")
        }
    },
    "extract_marks": load_prompt(prompt_dir / "extract_mark_count" / "extract_mark_count_prompt.json"),
    }

# Grade Answers for Each Student

## Pass 1: Grade all Student Answers with Few Shot Prompting

In [23]:
# Pass 1: Grade All Answers
for _, student_answer in enumerate(student_answers_records):
   prompt_template = prompts["grade_answer"].get(student_answer["subject_id"]).get(student_answer["question_type"])

   student_answer["start_time"] = time.time()
   try:

      # Grade Answer
      logging.info(f"Starting Grading Answer for Student ID: {student_answer['student_id']} and Question ID: {student_answer['question_id']}")

      input_args = {
         "question": student_answer["question_text"], 
         "answer": student_answer["answer_text"], 
         "mark_scheme": student_answer["mark_scheme_text"], 
         "context": student_answer["context"]
         }
      prompt_template_input_args = dict((k, input_args[k]) for k in prompt_template.input_variables if k in input_args)

      graded_answer_response = run_prompt_chain(
         prompt_template=prompt_template.template,
         llm=llm,
         burn_in_runs=2,
         **prompt_template_input_args
         )
      
      student_answer["llm_graded_answer"] = graded_answer_response["prompt_chain_response"]
      student_answer["llm_graded_answer_token_costing"] = graded_answer_response["prompt_chain_token_costing"]
      
      logging.info(f"Completed Grading Answer for Student ID: {student_answer['student_id']} and Question ID: {student_answer['question_id']}")
      
      # Extract Marks
      logging.info(f"Starting Extracting Marks for Student ID: {student_answer['student_id']} and Question ID: {student_answer['question_id']}")

      # try: 
      #    marks = extract_marks_from_graded_answer(student_answer["llm_graded_answer"])
      #    marking_token_cost = 0
      # except IndexError:
      #    extract_marks_response = run_prompt_chain(
      #          prompt_template=prompts["extract_marks"].template,
      #          llm=llm,
      #          burn_in_runs=1,
      #          answer=student_answer["llm_graded_answer"]
      #          )
      #    marks = extract_marks_response["prompt_chain_response"]
      #    marking_token_cost = extract_marks_response["prompt_chain_token_costing"]
      
      extract_marks_response = run_prompt_chain(
         prompt_template=prompts["extract_marks"].template,
         llm=llm,
         burn_in_runs=1,
         answer=student_answer["llm_graded_answer"]
         )
      marks = extract_marks_response["prompt_chain_response"]
      marking_token_cost = extract_marks_response["prompt_chain_token_costing"]

      student_answer["llm_awarded_marks"] = int(marks)
      student_answer["llm_awarded_marks_token_costing"] = marking_token_cost
      
      logging.info(f"Completed Extracting Marks for Student ID: {student_answer['student_id']} and Question ID: {student_answer['question_id']}")

   except:
      logging.info(f"Question type: {student_answer['question_type']} is not supported. Assigning NA and skipping to next question.")

      student_answer["llm_graded_answer"] = "NA"   
      student_answer["llm_graded_answer_token_costing"]
      student_answer["llm_awarded_marks"] = np.nan
      student_answer["llm_awarded_marks_token_costing"] = 0
   
   # Compute hitrate
   min_mark = min( [student_answer.get("awarded_marks")] )
   max_mark = max( [student_answer.get("awarded_marks")] )

   student_answer["llm_mark_hitrate"]=min_mark<=student_answer["llm_awarded_marks"]<=max_mark
   
   student_answer["end_time"] = time.time()
   student_answer["elapsed_time_in_seconds"] = student_answer["end_time"] - student_answer["start_time"]

   # Randomly sleep for seconds to avoid API throttling. Between 1-3 seconds
   seconds_to_sleep = random.sample([1,2,3], 1)[0]
   
   logging.info(f"Sleeping for {seconds_to_sleep} seconds to avoid API Throttling.")
   time.sleep(seconds_to_sleep)
      

2024-07-15 15:30:44,628-INFO-Starting Grading Answer for Student ID: 5000 and Question ID: 1.0
2024-07-15 15:30:44,676-INFO-Construct LLM Chain
2024-07-15 15:30:44,702-INFO-Run LLM Chain with 2 Burn in Run(s)
2024-07-15 15:31:23,133-INFO-Completed Grading Answer for Student ID: 5000 and Question ID: 1.0
2024-07-15 15:31:23,133-INFO-Starting Extracting Marks for Student ID: 5000 and Question ID: 1.0
2024-07-15 15:31:23,134-INFO-Construct LLM Chain
2024-07-15 15:31:23,135-INFO-Run LLM Chain with 1 Burn in Run(s)
2024-07-15 15:31:24,346-INFO-Completed Extracting Marks for Student ID: 5000 and Question ID: 1.0
2024-07-15 15:31:24,347-INFO-Sleeping for 2 seconds to avoid API Throttling.
2024-07-15 15:31:26,348-INFO-Starting Grading Answer for Student ID: 5000 and Question ID: 2.0
2024-07-15 15:31:26,349-INFO-Construct LLM Chain
2024-07-15 15:31:26,350-INFO-Run LLM Chain with 2 Burn in Run(s)
2024-07-15 15:31:53,035-INFO-Completed Grading Answer for Student ID: 5000 and Question ID: 2.0
2024

In [24]:
# Combine Student Answers and save as a DataFrame
completed_paper_df = pd.DataFrame(student_answers_records)

In [26]:
# Save graded student answers
completed_paper_df.to_csv(data_dir / "student_answers_llm_graded.csv", index=False)